# Code documentation Q&A bot example with LangChain
![picture](https://lancedb.github.io/lancedb/assets/ecosystem-illustration.png)

This Q&A bot will allow you to query your own documentation easily using questions. We'll also demonstrate the use of LangChain and LanceDB using the OpenAI API.

In this example we'll **Numpy 1.26** documentation, but, this could be replaced for your own docs as well

### Installing dependencies

In [5]:
! pip install -U langchain langchain-openai langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 68.5 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [2]:
! pip install -qq tiktoken unstructured pandas lancedb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 11.7 MB/s eta 0:00:00


First, let's get some setup out of the way. As we're using the OpenAI API, ensure that you've set your key (and organization if needed):

In [3]:
import openai
import os


# Configuring the environment variable OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = "sk-proj-...."

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "sk-..."
openai.api_key = os.environ["OPENAI_API_KEY"]

# assert len(openai.models.list()["data"]) > 0


We're going to use the power of LangChain to help us create our Q&A bot. It comes with several APIs that can make our development much easier as well as a LanceDB integration for vectorstore.

### Importing all libraries

In [6]:
import lancedb
import re
import pickle
import requests
import zipfile
from pathlib import Path

from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import LanceDB
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.chains import RetrievalQA

To make this easier, we've downloaded Pandas documentation and stored the raw HTML files for you to download. We'll download them and then use LangChain's HTML document readers to parse them and store them in LanceDB as a vector store, along with relevant metadata.
By default we'll use numpy docs as it is much smaller than pandas docs, but you can replace this with your own docs as well.

### Get the data

In [7]:
# pandas_docs = requests.get("https://eto-public.s3.us-west-2.amazonaws.com/datasets/pandas_docs/pandas.documentation.zip")

numpy_docs = requests.get("https://numpy.org/doc/1.26/numpy-html.zip")
with open("numpy-html.zip", "wb") as f:
    f.write(numpy_docs.content)

file = zipfile.ZipFile("numpy-html.zip")
file = file.extractall(path="numpy_docs")

We'll create a simple **helper function** that can help to extract metadata, so we can use this downstream when we're wanting to query with filters. In this case, we want to keep the lineage of the uri or path for each document that we process:

In [8]:
# Pre-processing and loading the documentation

# Next, let's pre-process and load the documentation. To make sure we don't need to do this repeatedly if we were updating code,
# we're caching it using pickle so we can retrieve it again (this could take a few minutes to run the first time you do it).
# We'll also add some more metadata to the docs here such as the title and version of the code:
import re


def get_document_title(document_list):
    titles = []
    for doc in document_list:
        if "metadata" in doc and "source" in doc["metadata"]:
            m = str(doc["metadata"]["source"])
            title = re.findall("numpy_docs(.*).html", m)
            print(title)
            if title:
                titles.append(title[0])
            else:
                titles.append("")
        else:
            titles.append("")
    return titles

# Pre-processing and loading the documentation

Next, let's pre-process and load the documentation. To make sure we don't need to do this repeatedly if we were updating code, we're caching it using pickle so we can retrieve it again (this could take a few minutes to run the first time you do it). We'll also add some more metadata to the docs here such as the title and version of the code:

If there is some issue with nltk package, kindly try using
```
import nltk
nltk.download('punkt')
```
or try to manually install the [nltk_data](https://github.com/nltk/nltk_data/tree/gh-pages) package and unzip the **punkt tokenizer** zip and the **averaged_perceptron_tagger** zip file in the packages folder.

In [11]:
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
from tqdm import tqdm

docs = []
docs_path = Path("docs.pkl")
for p in tqdm(Path("numpy_docs").rglob("*.html")):
    if p.is_dir():
        continue
    print(p)
    loader = UnstructuredHTMLLoader(str(p))
    raw_document = loader.load()
    # docs.append(raw_document)
    title = get_document_title(raw_document)
    m = {"title": title}
    if raw_document:
        raw_document[0].metadata.update(m)
        raw_document[0].metadata["source"] = str(raw_document[0].metadata["source"])
        docs.extend(raw_document)


if docs:
    with open(docs_path, "wb") as fh:
        pickle.dump(docs, fh)
else:
    with open(docs_path, "rb") as fh:
        docs = pickle.load(fh)

0it [00:00, ?it/s]

numpy_docs/index.html


1it [00:03,  3.51s/it]

numpy_docs/benchmarking.html


2it [00:04,  1.75s/it]

...

numpy_docs/reference/c-api/data_memory.html


2696it [08:20,  2.42it/s]

numpy_docs/reference/c-api/array.html


2699it [08:21,  5.38it/s]


numpy_docs/reference/distutils/misc_util.html
numpy_docs/_static/webpack-macros.html


In [15]:
len(docs)

2699

# Generating emebeddings from our docs

Now that we have our raw documents loaded, we need to pre-process them to generate embeddings:

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()

# Storing

Let's connect to LanceDB so we can store our documents, It requires 0 setup !

### For LanceDB cloud use the below cell :

In [ ]:
vectorstore = LanceDB(
    embedding=embeddings,
    uri="db://test",  # your remote database URI
    api_key="sk_...",
    region="us-east-x-xxx",  # the cloud region you have configured
    table_name="langchain_vectorstore",  # Optional, defaults to "vectors"
    mode="overwrite",  # Optional, defaults to "overwrite"
)

doc_ids = vectorstore.add_documents(documents=documents)

### For LanceDB local use the below cell :

In [18]:
vectorstore = LanceDB.from_documents(documents=documents, embedding=embeddings)

Now let's create our RetrievalQA chain using the LanceDB vector store:

In [19]:
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(), chain_type="stuff", retriever=vectorstore.as_retriever()
)

And thats it! We're all setup. The next step is to run some queries, let's try a few:

### Querying

In [20]:
query = "tell me about the numpy library?"
qa.invoke(query)

{'query': 'tell me about the numpy library?',
 'result': ' The NumPy library is a Python library that provides multidimensional array and matrix data structures, as well as various routines for efficient operations on these structures. It is commonly used for scientific computing and has a vast library of mathematical functions. It also has a fixed array size and is used extensively in other popular data science packages such as Pandas and scikit-learn.'}

In [21]:
query = "What's the current version of numpy?"
qa.invoke(query)

{'query': "What's the current version of numpy?",
 'result': ' The current version of NumPy is 1.26.'}

In [22]:
query = "What kind of linear algebra related operations can be done in numpy?"
qa.invoke(query)

{'query': 'What kind of linear algebra related operations can be done in numpy?',
 'result': "\n\nSome possible operations that can be performed using numpy's linear algebra functions include matrix multiplication, finding eigenvalues and eigenvectors, calculating determinants and inverses, solving systems of linear equations, and performing decompositions such as LU, QR, and SVD."}

Thanks !